In [1]:
import pandas as pd
import numpy as np

import os
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error

import torch
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from haversine import haversine, Unit

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
# --- 0. Configuration and Constants ---
SEQUENCE_SIZE = 48  # Sequence length for PINN input
TARGET_WINDOW = 24  # Forecast horizon
TYPHOON_STEPS_IN = 12 # Sequence length for Seq2Seq input
PROXIMITY_THRESHOLD_KM = 600 # Filter for historical data

STATIONS = ['Guanyin', 'Keelung', 'Longtan', 'Taipei', 'Tamsui', 'Taoyuan', 'Yangmingshan']
station_name = STATIONS[4]
device = torch.device('cpu')

# Feature Definitions
WEATHER_FEATURES = ['air_pressure', 'temperature', 'relative_humidity', 'wind_speed', 
                    'wind_direction', 'gust_max', 'gust_max_dir', 'precipitation', 'solar_rad'] # 9 features
TARGET_FEATURE = ['wind_speed'] 
TYPHOON_FEATURES_CORE = ['lat', 'lng', 'wind', 'long50'] # 4 core features for Seq2Seq
TYPHOON_FEATURES_EXO_PRED = ['typhoon_lat', 'typhoon_lng', 'wind', 'long50', 'distance_km'] # 5 exogenous features for PINN

# Full feature set for PINN input (9 weather + 5 exo = 14 features)
PINN_ALL_FEATURES = WEATHER_FEATURES + TYPHOON_FEATURES_EXO_PRED

In [3]:
# --- 1. Model Definitions ---

class Seq2Seq(nn.Module):
    # Model structure provided by user
    def __init__(self, input_size, hidden_size, num_layers, output_size, num_heads=1):
        super(Seq2Seq, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.encoder = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.multihead_attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=num_heads, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, horizon_length):
        encoder_output, (hidden, cell) = self.encoder(x)
        decoder_input = torch.zeros(x.size(0), horizon_length, self.hidden_size, device=x.device)
        decoder_out, _ = self.decoder(decoder_input, (hidden, cell))
        attention_output, _ = self.multihead_attention(query=decoder_out, key=encoder_output, value=encoder_output)
        final_output = self.fc(attention_output)
        return final_output

class HybridModelPINN(nn.Module):
    # Model structure provided by user
    def __init__(self, input_size, output_size, sequence_length,
                 embedding_dim=48, n_head=4, num_transformer_layers=4,
                 conv_channels=64, kernel_size=3, pool_kernel=2,
                 hidden_dense=512):
        super(HybridModelPINN, self).__init__()
        self.input_size = input_size 
        self.output_size = output_size
        self.sequence_length = sequence_length
        self.embedding_dim = embedding_dim
        
        self.linear_projection = nn.Linear(input_size, embedding_dim)
        self.positional_encoding = self._get_positional_encoding(sequence_length, embedding_dim)
        self.register_buffer('positional_encoding_buffer', self.positional_encoding)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=n_head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=num_transformer_layers)
        self.conv_layer = nn.Conv1d(embedding_dim, conv_channels, kernel_size, padding='same')
        self.relu = nn.ReLU()
        self.pooling_layer = nn.MaxPool1d(pool_kernel)

        self.conv_output_length = (sequence_length + 2 * (kernel_size // 2) - (kernel_size - 1) - 1) + 1
        self.pooled_output_length = self.conv_output_length // pool_kernel

        self.dense1 = nn.Linear(conv_channels * self.pooled_output_length + embedding_dim * sequence_length, hidden_dense) 
        self.dense2 = nn.Linear(hidden_dense, output_size)
        self.u_dense = nn.Linear(hidden_dense, output_size)
        self.v_dense = nn.Linear(hidden_dense, output_size)
        self.p_dense = nn.Linear(hidden_dense, output_size)

    def _get_positional_encoding(self, seq_len, d_model):
        position = torch.arange(seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe = torch.zeros(seq_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        return pe.squeeze(1).unsqueeze(0).transpose(1, 2) 

    def forward(self, x):
        embedded = self.linear_projection(x)
        positional_encoding = self.positional_encoding_buffer[:, :embedded.size(1), :].transpose(1, 2) 
        embedded = embedded + positional_encoding.squeeze(2)

        transformer_out = self.transformer_encoder(embedded)
        transformer_out_flattened = transformer_out.view(transformer_out.size(0), -1)

        cnn_in = embedded.permute(0, 2, 1)
        conv_out = self.relu(self.conv_layer(cnn_in))
        pooled_out = self.pooling_layer(conv_out)
        pooled_out_flattened = pooled_out.view(pooled_out.size(0), -1)

        combined_features = torch.cat((pooled_out_flattened, transformer_out_flattened), dim=1)
        dense1_out = self.relu(self.dense1(combined_features))

        output = self.dense2(dense1_out)
        u_out = self.u_dense(dense1_out)
        v_out = self.v_dense(dense1_out)
        p_out = self.p_dense(dense1_out)

        return output, u_out, v_out, p_out

In [4]:
# --- 2. Utility Functions (Includes User's PINN Loss & Sequence Creation) ---

def calculate_distance(row):    
    try:
        typhoon_loc = (row['lat'], row['lng'])
    except:
        typhoon_loc = (row['typhoon_lat'], row['typhoon_lng'])
    
    try:
        station_loc = (row['station_latitude'], row['station_longitude'])
    except:
        station_loc = (row['latitude'], row['longitude'])
        
    if pd.isnull(typhoon_loc[0]) or pd.isnull(station_loc[0]):
        return np.nan

    try:
        return haversine(typhoon_loc, station_loc, unit=Unit.KILOMETERS)
    except ValueError as e:
        corrected_longitude = typhoon_loc[1]
        if corrected_longitude > 180: corrected_longitude = 180
        elif corrected_longitude < -180: corrected_longitude = -180
        corrected_typhoon_loc = (typhoon_loc[0], corrected_longitude)
        return haversine(corrected_typhoon_loc, station_loc, unit=Unit.KILOMETERS)

def create_sequences_typhoon(data_values, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(data_values)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(data_values):
            break
        seq_x = data_values[i:end_ix]
        seq_y = data_values[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def create_sequence_weather(sequence_length, target_window, scaled_data_df):
    target_column_name = 'wind_speed'
    target_column_index = scaled_data_df.columns.get_loc(target_column_name)
    feature_indices = [scaled_data_df.columns.get_loc(col) for col in scaled_data_df.columns if col != 'time']
    
    x = []
    y = []
    for i in tqdm(range(len(scaled_data_df) - sequence_length - target_window + 1), desc="Creating Sequences"):
        sequence = scaled_data_df.iloc[i:i+sequence_length, feature_indices].values
        target = scaled_data_df.iloc[i+sequence_length:i+sequence_length+target_window, target_column_index].values
        x.append(sequence)
        y.append(target)
    
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

def physics_loss_fn(u, v, p, x):
    # Stub function for physics loss in this environment
    return torch.tensor(0.0) 

def run_seq2seq_training_and_forecast(typhoon_df, model_config, scaler_typhoon):
    print("   -> Starting Seq2Seq Training and Forecasting...")
    # 1. Prepare sequences for Seq2Seq training
    X_typhoon, y_typhoon = create_sequences_typhoon(typhoon_df[TYPHOON_FEATURES_CORE].values, TYPHOON_STEPS_IN, TARGET_WINDOW)
    
    # First split: 70% train, 30% temp (which will be split into val + test)
    split_idx = int(0.7 * len(X_typhoon))
    X_train, X_temp = X_typhoon[:split_idx], X_typhoon[split_idx:]
    y_train, y_temp = y_typhoon[:split_idx], y_typhoon[split_idx:]

    # Second split: split the temp set into 50% validation and 50% test
    val_split_idx = int(0.5 * len(X_temp))

    X_valid, X_test = X_temp[:val_split_idx], X_temp[val_split_idx:]
    y_valid, y_test = y_temp[:val_split_idx], y_temp[val_split_idx:]



    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train).float()
    
    X_valid_tensor = torch.from_numpy(X_valid).float()
    y_valid_tensor = torch.from_numpy(y_valid).float()
    
    X_test_tensor = torch.from_numpy(X_test).float()
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
    
    # Step 3: Train the model (Simulated: 1 batch)
    typhoon_model = Seq2Seq(**model_config)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(typhoon_model.parameters(), lr=0.001)
    
    
    epochs = 1
    early_stop_count = 0  
    min_val_loss = float('inf')  # Initialize with a high value
    val_loss_history = []
    for epoch in tqdm(range(epochs), desc="Typhoon Model Training & Validation"):
        typhoon_model.train()
        for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Train)"):
            optimizer.zero_grad()
            output = typhoon_model(batch_x, TARGET_WINDOW)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
        typhoon_model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{epochs} (Validation)"):
                x_batch, y_batch = batch
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = typhoon_model(x_batch, TARGET_WINDOW)
                loss = criterion(outputs, y_batch)
                val_losses.append(loss.item())
        val_loss = np.mean(val_losses)
        val_loss_history.append(val_loss)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            early_stop_count = 0
        else:
            early_stop_count += 1
            print(f'[LOG] Early Stopping Counter: {early_stop_count}')

        if early_stop_count >= 10:
            print(f"[LOG] Early stopping at epoch {epoch + 1}")
            break
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")
                           
    # Step 4: Save the model
    torch.save(typhoon_model.state_dict(), 'models/typhoon/typhoon_model.pth')
    print("   -> Saved 'typhoon_model.pth'")
    
    # Step 5: Generate forecast for all test sequences
    typhoon_model.eval()
    all_predictions = []
    test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=64, shuffle=False)
    
    with torch.no_grad():
        for batch_x_test in test_loader:
            output = typhoon_model(batch_x_test[0], TARGET_WINDOW).cpu().numpy()
            all_predictions.append(output)
            
    raw_predictions = np.concatenate(all_predictions, axis=0)

    # Inverse transform the predictions 
    predicted_physical_output = scaler_typhoon.inverse_transform(raw_predictions.reshape(-1, len(TYPHOON_FEATURES_CORE))).reshape(raw_predictions.shape)
    
    # Step 6: Create the full exogenous data (Actual input dates + Forecasted dates)
    full_exogenous_df_list = []
    
    # The time indices for the input sequences of the test set
    test_indices = np.arange(split_idx, len(typhoon_df) - TYPHOON_STEPS_IN - TARGET_WINDOW + 1)
    
    for i in range(raw_predictions.shape[0]):
        # Get the time index corresponding to the start of the current test sequence
        current_sequence_start_index = split_idx + i 
        
        # This part of the data is the historical data for the input sequence itself
        df_input = typhoon_df.iloc[current_sequence_start_index : current_sequence_start_index + TYPHOON_STEPS_IN].reset_index(drop=True)
        
        # This part is the predicted future data
        input_end_time = typhoon_df['time'].iloc[current_sequence_start_index + TYPHOON_STEPS_IN - 1]
        forecast_times = pd.date_range(start=input_end_time + pd.Timedelta(hours=1), periods=TARGET_WINDOW, freq='H')
        
        df_pred = pd.DataFrame(predicted_physical_output[i], columns=TYPHOON_FEATURES_CORE)
        df_pred['time'] = forecast_times
        df_pred['sequence_id'] = i  # Identifier for the sequence the forecast belongs to
        
        # Combine input sequence (historical) + forecast sequence (predicted)
        df_full_seq = pd.concat([df_input, df_pred], ignore_index=True)
        df_full_seq['sequence_id'] = i
        full_exogenous_df_list.append(df_full_seq)
        
    return pd.concat(full_exogenous_df_list, ignore_index=True)


def calculate_metrics(y_true_scaled, y_pred_scaled, scaler_target):
    # Flatten and Inverse Transform for R2 calculation on physical units
    
    # y_true_scaled is multi-step output, needs to be flattened: (N, 24) -> (N*24, 1)
    y_true_flat_scaled = y_true_scaled.flatten().reshape(-1, 1)
    y_pred_flat_scaled = y_pred_scaled.flatten().reshape(-1, 1)
    
    # Inverse transform to get true physical units
    y_true = scaler_target.inverse_transform(y_true_flat_scaled)
    y_pred = scaler_target.inverse_transform(y_pred_flat_scaled)

    # Calculate Metrics
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    return {
        "R2": r2,
        "RMSE": rmse,
        "MSE": mse,
        "MAE": mae
    }

def run_pinn_training(weather_df, typhoon_data_hist_raw, station_coords, model_config):
    print(f"   -> Starting PINN Training for {station_name}...")
    
    # 1. Load weather station data (done before call)
    station_coords.rename(columns={'latitude': 'station_latitude', 
                                   'longitude': 'station_longitude', 
                                   'location': 'station'}, inplace=True)
    
    weather_df = pd.merge(weather_df, station_coords[['station', 'station_latitude', 'station_longitude']], on='station', how='left')
    
    # 2. Filter exogenous data: Combine historical weather with historical typhoon data
    weather_exo_merge = pd.merge_asof(
        weather_df.sort_values('time'),
        typhoon_data_hist_raw[['time'] + TYPHOON_FEATURES_CORE].sort_values('time'),
        on='time',
        direction='nearest'
    )
    
    weather_exo_merge.rename(columns={'lat': 'typhoon_lat', 'lng': 'typhoon_lng'}, inplace=True)
    weather_exo_merge['distance_km'] = weather_exo_merge.apply(calculate_distance, axis=1)
    
    # Apply 600km proximity filter: 
    weather_exo_merge['is_affected'] = (weather_exo_merge['distance_km'] <= PROXIMITY_THRESHOLD_KM).astype(int)
    
    # Create the final exogenous feature columns (5 features)
    weather_exo_merge[TYPHOON_FEATURES_EXO_PRED] = weather_exo_merge[['typhoon_lat', 'typhoon_lng', 'wind', 'long50', 'distance_km']].copy()
    
    # 3. Combine weather station data and filtered exogenous data. Fill non-exogenous periods with 0.
    final_pinn_df = weather_exo_merge[WEATHER_FEATURES + TYPHOON_FEATURES_EXO_PRED + ['time']].copy()
    
    # Set exogenous features to 0 for periods NOT affected by the typhoon (< 600km)
    final_pinn_df.loc[weather_exo_merge['is_affected'] == 0, TYPHOON_FEATURES_EXO_PRED] = 0
    
    # Scaling (for PINN)
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()
    
    # Scale all 14 features
    final_pinn_df[PINN_ALL_FEATURES] = scaler_features.fit_transform(final_pinn_df[PINN_ALL_FEATURES])
    final_pinn_df[TARGET_FEATURE] = scaler_target.fit_transform(final_pinn_df[TARGET_FEATURE])
    
    # Prepare Sequences and DataLoaders for PINN training
    X_pinn, y_pinn = create_sequence_weather(SEQUENCE_SIZE, TARGET_WINDOW, final_pinn_df)
    
    X_train, X_temp, y_train, y_temp = train_test_split(X_pinn, y_pinn, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
    
    # Step 4: Train the model (Simulated: 1 batch)
    model_pinn = HybridModelPINN(**model_config)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model_pinn.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, 
                              patience=3)
    
    # Training Loop
    epochs = 1
    early_stop_count = 0
    min_val_loss = float('inf')
    train_loss_history = []
    val_loss_history = []
    physics_loss_history = []  # Store physics loss

    for epoch in tqdm(range(epochs), desc="Physics-Regulated Model Training and Validation"):
        model_pinn.train()
        train_loss = []
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Train)"):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs, u_pred, v_pred, p_pred = model_pinn(x_batch) #get the u,v,p predictions

            # Data loss
            data_loss = criterion(outputs, y_batch)

            # Physics loss
            physics_loss = physics_loss_fn(u_pred, v_pred, p_pred, x_batch)  # Pass x_batch

            # Combine the losses with weights 0.73 and 0.27
            loss = 0.73 * data_loss + 0.27 * physics_loss

            train_loss.append(loss.item())
            physics_loss_history.append(physics_loss.item()) #store physics loss
            loss.backward()
            optimizer.step()

        train_loss_history.append(np.mean(train_loss))

        # Validation
        model_pinn.eval()
        val_losses = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} (Validation)"):
                x_batch, y_batch = batch
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs, _, _, _ = model_pinn(x_batch)  # No u,v,p for validation
                loss = criterion(outputs, y_batch)
                val_losses.append(loss.item())

        val_loss = np.mean(val_losses)
        val_loss_history.append(val_loss)
        scheduler.step(val_loss)

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            early_stop_count = 0
        else:
            early_stop_count += 1
            print(f'[LOG] Early Stopping Counter: {early_stop_count}')

        if early_stop_count >= 10:
            print(f"[LOG] Early stopping at epoch {epoch + 1}")
            break
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")
    
    model_pinn.eval()
    X_test_tensor = X_test.to(device)
    
    with torch.no_grad():
        y_pred_tensor, _, _, _ = model_pinn(X_test_tensor)
        y_pred_scaled = y_pred_tensor.cpu().numpy()
    metrics = calculate_metrics(y_test.numpy(), y_pred_scaled, scaler_target)
    
    # Step 5: Save the model
    torch.save(model_pinn.state_dict(), f'models/weather_sta/{station_name.lower()}_pinn_model.pth')
    print(f"   -> Saved '{station_name.lower()}_pinn_model.pth'")

    return weather_exo_merge['is_affected'].sum(), metrics

In [5]:
# 1. Load Typhoon Data
typhoon_data_hist_raw = pd.read_csv('data/typhoon_data.csv', parse_dates=['Date'], infer_datetime_format=True)
typhoon_data_hist_raw.rename(columns={'Date': 'time'}, inplace=True)

# 2. Filter Typhoon Data to >= 2015
typhoon_data_hist_filtered = typhoon_data_hist_raw[(typhoon_data_hist_raw['time'].dt.year >= 2015)].copy()
typhoon_data_hist_filtered = typhoon_data_hist_filtered[TYPHOON_FEATURES_CORE + ['time']].copy()
typhoon_data_hist_filtered.sort_values(by='time', inplace=True)
typhoon_data_hist_filtered.dropna(inplace=True)

# Scale the filtered typhoon data for training
scaler_typhoon = MinMaxScaler(feature_range=(0, 1))
typhoon_data_scaled_for_training = typhoon_data_hist_filtered.copy()
typhoon_data_scaled_for_training[TYPHOON_FEATURES_CORE] = scaler_typhoon.fit_transform(typhoon_data_hist_filtered[TYPHOON_FEATURES_CORE])

In [6]:
print("--- 颱風模式訓練 (Typhoon Model Training: Seq2Seq) ---")
# 3, 4, 5, 6: Train, Save, Forecast, Generate Exogenous Data
seq2seq_config = {
    "input_size": len(TYPHOON_FEATURES_CORE), 
    "hidden_size": 128, 
    "num_layers": 2, 
    "output_size": len(TYPHOON_FEATURES_CORE), 
    "num_heads": 4
}

full_forecast_exogenous_df = run_seq2seq_training_and_forecast(
    typhoon_data_scaled_for_training, seq2seq_config, scaler_typhoon
)

print("\n✅ Typhoon Model Training & Forecasting Complete.")

--- 颱風模式訓練 (Typhoon Model Training: Seq2Seq) ---
   -> Starting Seq2Seq Training and Forecasting...


Typhoon Model Training & Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1 (Train):   0%|          | 0/442 [00:00<?, ?it/s]

Epoch 1/1 (Validation):   0%|          | 0/95 [00:00<?, ?it/s]

Epoch 1/1, Validation Loss: 0.0190
   -> Saved 'typhoon_model.pth'

✅ Typhoon Model Training & Forecasting Complete.


In [7]:
# --- Individual Weather Station Workflow ---
print("\n--- 個別氣象站模型訓練 (Individual Station Training: Hybrid PINN) ---")

# Load data
station_coords = pd.read_csv('data/weather_station_coords.csv')
station_coords.rename(columns={'lat': 'station_latitude', 'long': 'station_longitude', 'location': 'station'}, inplace=True)
weather_sta_df = pd.read_csv(f'data/indiv_weather_station/{station_name}.csv').fillna(0)

# Process data
time_col = [col for col in weather_sta_df.columns if 'time' in col.lower()][0]
weather_sta_df['time'] = pd.to_datetime(weather_sta_df[time_col], format='mixed', dayfirst=True)
weather_sta_df.columns = [f'time_{station_name}', 'air_pressure', 'temperature',
                      'relative_humidity', 'wind_speed',
                      'wind_direction', 'gust_max', 'gust_max_dir',
                      'precipitation', 'solar_rad', 'time']
weather_sta_df.drop(columns=[f'time_{station_name}'], inplace=True)
weather_sta_df = weather_sta_df[(weather_sta_df['time'].dt.year >= 2015)].copy()
weather_sta_df['station'] = station_name

pinn_config = {
    "input_size": len(PINN_ALL_FEATURES), 
    "output_size": TARGET_WINDOW, 
    "sequence_length": SEQUENCE_SIZE, 
    "hidden_dense": 512
}

affected_count, pinn_metrics = run_pinn_training(weather_sta_df[-10000:], typhoon_data_hist_raw, station_coords, pinn_config)

print(f"\n✅ Individual Station Model Training Complete for {station_name}.")
print(f"Total historical typhoon instances within {PROXIMITY_THRESHOLD_KM}km threshold: {affected_count} records.")

# --- Metric Scores ---
print("\n--- Hybrid PINN Model Metric Scores (Test Set) ---")
print(f"R2 Score (R2): {pinn_metrics['R2']:.6f}")

print(f"Root Mean Squared Error (RMSE): {pinn_metrics['RMSE']:.6f}")
print(f"Mean Squared Error (MSE): {pinn_metrics['MSE']:.6f}")
print(f"Mean Absolute Error (MAE): {pinn_metrics['MAE']:.6f}")

print("\n--- Saved Models (Simulated) ---")
print("typhoon_model.pth (Typhoon Track Forecasting Model)")
print(f"{station_name.lower()}_pinn_model.pth (Weather Station Wind Speed Model)")


--- 個別氣象站模型訓練 (Individual Station Training: Hybrid PINN) ---
   -> Starting PINN Training for Tamsui...


Creating Sequences:   0%|          | 0/9929 [00:00<?, ?it/s]

Physics-Regulated Model Training and Validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1 (Train):   0%|          | 0/497 [00:00<?, ?it/s]

Epoch 1/1 (Validation):   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1/1, Validation Loss: 0.0011
   -> Saved 'tamsui_pinn_model.pth'

✅ Individual Station Model Training Complete for Tamsui.
Total historical typhoon instances within 600km threshold: 189 records.

--- Hybrid PINN Model Metric Scores (Test Set) ---
R2 Score (R2): -7.107145
Root Mean Squared Error (RMSE): 0.031318
Mean Squared Error (MSE): 0.000981
Mean Absolute Error (MAE): 0.027842

--- Saved Models (Simulated) ---
typhoon_model.pth (Typhoon Track Forecasting Model)
tamsui_pinn_model.pth (Weather Station Wind Speed Model)


In [8]:
print(f"\n✅ Individual Station Model Training Complete for {station_name}.")
print(f"Total historical typhoon instances within {PROXIMITY_THRESHOLD_KM}km threshold: {affected_count} records.")

# --- Metric Scores ---
print("\n--- Hybrid PINN Model Metric Scores (Test Set) ---")
print(f"Root Mean Squared Error (RMSE): {pinn_metrics['RMSE']:.6f}")
print(f"Mean Squared Error (MSE): {pinn_metrics['MSE']:.6f}")
print(f"Mean Absolute Error (MAE): {pinn_metrics['MAE']:.6f}")

print("\n--- Saved Models (Simulated) ---")
print("typhoon_model.pth (Typhoon Track Forecasting Model)")
print(f"{station_name.lower()}_pinn_model.pth (Weather Station Wind Speed Model)")


✅ Individual Station Model Training Complete for Tamsui.
Total historical typhoon instances within 600km threshold: 189 records.

--- Hybrid PINN Model Metric Scores (Test Set) ---
Root Mean Squared Error (RMSE): 0.031318
Mean Squared Error (MSE): 0.000981
Mean Absolute Error (MAE): 0.027842

--- Saved Models (Simulated) ---
typhoon_model.pth (Typhoon Track Forecasting Model)
tamsui_pinn_model.pth (Weather Station Wind Speed Model)


In [9]:
typhoon_data_hist_raw[-100:]

,time,grade,lat,lng,pressure,wind,dir50,long50,short50,dir30,long30,short30,intp,seq_id
222447,2022-11-13 04:00:00,3,22.06,165.82,1004.0,35.0,0,0,0,1,180,90,1,202224
222448,2022-11-13 05:00:00,3,22.08,165.81,1004.0,35.0,0,0,0,1,180,90,1,202224
222449,2022-11-13 06:00:00,3,22.10,165.80,1004.0,35.0,0,0,0,1,180,90,0,202224
222450,2022-11-13 07:00:00,3,22.11,165.79,1004.0,35.0,0,0,0,1,180,90,1,202224
222451,2022-11-13 08:00:00,3,22.12,165.78,1004.0,35.0,0,0,0,1,180,90,1,202224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222542,2022-12-12 08:00:00,3,20.40,131.30,1003.3,35.0,0,0,0,8,150,90,1,202225
222543,2022-12-12 09:00:00,3,20.38,131.38,1004.0,35.0,0,0,0,8,150,90,1,202225
222544,2022-12-12 10:00:00,3,20.36,131.44,1004.7,35.0,0,0,0,8,150,90,1,202225
222545,2022-12-12 11:00:00,3,20.33,131.47,1005.3,35.0,0,0,0,8,150,90,1,202225


In [10]:
print(f"Historical typhoon instances within {PROXIMITY_THRESHOLD_KM}km threshold from {station_name}: {affected_count} records.")

Historical typhoon instances within 600km threshold from Tamsui: 189 records.


In [11]:
print(f"\n✅ Individual Station Model Training Complete for {station_name}.")

try:
    pinn_metrics['RMSE'] = pinn_metrics['RMSE'].item()
except:
    pass
pd.DataFrame([pinn_metrics]).to_csv(f'models/weather_sta/metadata/{station_name}_metrics.csv')

# --- Metric Scores ---
print("\n--- Hybrid PINN Model Metric Scores (Test Set) ---")
print(f"R2 Score (R2): {pinn_metrics['R2']:.6f}")

print(f"Root Mean Squared Error (RMSE): {pinn_metrics['RMSE']:.6f}")
print(f"Mean Squared Error (MSE): {pinn_metrics['MSE']:.6f}")
print(f"Mean Absolute Error (MAE): {pinn_metrics['MAE']:.6f}")

print("\n--- Saved Models (Simulated) ---")
print("typhoon_model.pth (Typhoon Track Forecasting Model)")
print(f"{station_name.lower()}_pinn_model.pth (Weather Station Wind Speed Model)")
print(f"Metrics Scores for {station_name} is saved!")


✅ Individual Station Model Training Complete for Tamsui.

--- Hybrid PINN Model Metric Scores (Test Set) ---
R2 Score (R2): -7.107145
Root Mean Squared Error (RMSE): 0.031318
Mean Squared Error (MSE): 0.000981
Mean Absolute Error (MAE): 0.027842

--- Saved Models (Simulated) ---
typhoon_model.pth (Typhoon Track Forecasting Model)
tamsui_pinn_model.pth (Weather Station Wind Speed Model)
Metrics Scores for Tamsui is saved!
